In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

2023-09-08 14:01:00.319713: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-08 14:01:00.332360: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 14:01:00.435149: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-08 14:01:00.435885: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-08 14:01:01.241261: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
df = pd.read_csv('smsspamcollection.tsv',delimiter='\t',encoding='latin-1')
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [3]:
X = df.message
Y = df.label
le = LabelEncoder()
Y = le.fit_transform(Y)

In [4]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [5]:
#Tokenize thx data and convert the text to sequences.
#Add padding to ensure that all the sequences have the same shape.
#There are many ways of taking the max_len and here an arbitrary length of 150 is chosen.
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [6]:
def RNN():
 inputs = Input(name='inputs',shape=[max_len])
 layer = Embedding(max_words,50,input_length=max_len)(inputs)
 layer = LSTM(64)(layer)
 layer = Dense(256,name='FC1')(layer)
 layer = Activation('relu')(layer)
 layer = Dropout(0.5)(layer)
 layer = Dense(1,name='out_layer')(layer)
 layer = Activation('sigmoid')(layer)
 model = Model(inputs=inputs,outputs=layer)
 return model

In [7]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [8]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
 validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
30/30 [==============================] - 6s 165ms/step - loss: 0.3952 - accuracy: 0.8625 - val_loss: 0.2561 - val_accuracy: 0.8734
Epoch 2/10
30/30 [==============================] - 2s 61ms/step - loss: 0.1708 - accuracy: 0.9504 - val_loss: 0.1074 - val_accuracy: 0.9694
Epoch 3/10
30/30 [==============================] - 2s 60ms/step - loss: 0.0734 - accuracy: 0.9810 - val_loss: 0.0720 - val_accuracy: 0.9810
Epoch 4/10
30/30 [==============================] - 2s 61ms/step - loss: 0.0523 - accuracy: 0.9868 - val_loss: 0.0560 - val_accuracy: 0.9821
Epoch 5/10
30/30 [==============================] - 2s 64ms/step - loss: 0.0368 - accuracy: 0.9900 - val_loss: 0.0599 - val_accuracy: 0.9842


In [10]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [11]:
loss,accr=model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 0s 15ms/step - loss: 0.0531 - accuracy: 0.9892


In [12]:
print("loss=",loss)
print("Accuracy=",accr)

loss= 0.05308206379413605
Accuracy= 0.989234447479248
